<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Spreadsheet-widget-for-the-Jupyter-Notebook" data-toc-modified-id="Spreadsheet-widget-for-the-Jupyter-Notebook-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Spreadsheet widget for the Jupyter Notebook</a></span><ul class="toc-item"><li><span><a href="#Installation" data-toc-modified-id="Installation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Installation</a></span></li></ul></li><li><span><a href="#Getting-started" data-toc-modified-id="Getting-started-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Getting started</a></span></li><li><span><a href="#Events" data-toc-modified-id="Events-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Events</a></span></li><li><span><a href="#Cell-ranges" data-toc-modified-id="Cell-ranges-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Cell ranges</a></span><ul class="toc-item"><li><span><a href="#Calculations" data-toc-modified-id="Calculations-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Calculations</a></span></li></ul></li><li><span><a href="#Renderers" data-toc-modified-id="Renderers-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Renderers</a></span></li><li><span><a href="#Plot" data-toc-modified-id="Plot-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Plot</a></span></li><li><span><a href="#Numpy-back-and-forth" data-toc-modified-id="Numpy-back-and-forth-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Numpy back and forth</a></span></li><li><span><a href="#pandas-back-and-forth" data-toc-modified-id="pandas-back-and-forth-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>pandas back and forth</a></span></li><li><span><a href="#Cell-widget" data-toc-modified-id="Cell-widget-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Cell widget</a></span></li><li><span><a href="#format" data-toc-modified-id="format-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>format</a></span></li></ul></div>

# Spreadsheet widget for the Jupyter Notebook
## Installation

With conda:

```
$ conda install -c conda-forge ipysheet
```

With pip:

```
$ pip install ipysheet
```

To make it work for Jupyter lab:
```
$ jupyter labextension install ipysheet
```

If you have notebook 5.2 or below, you also need to execute:
```
$ jupyter nbextension enable --py --sys-prefix ipysheet
```

# Getting started
Although ipysheet contains an object oriented interface, we recomment using the "state machine" based interface, similar to matplotlib's pyplot/pylab interface. Comparible to matplotlib pylab interface, this interface keeps track of the current sheet. Using the [cell](api.rst#ipysheet.easy.cell) function, [Cell](api.rst#ipysheet.sheet.Cell) widgets are added to the current sheet.

Importing ipysheet and invoking the [sheet](api.rst#ipysheet.easy.sheet) function will create the default spreadsheet widget. The function returns a [Sheet](api.rst#ipysheet.sheet.Sheet) instance, leaving that expression as a last statement of a code cell will display it, otherwise use `display(sheet)`.

Note that this documentation is a Jupyter notebook, and you can try it out directly on Binder:
[![Binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/QuantStack/ipysheet/master?filepath=docs%2Fsource%2Findex.ipynb)

In [1]:
import ipysheet
sheet = ipysheet.sheet()
sheet

Sheet(columns=5, layout=Layout(height='auto', width='auto'), rows=5)

Using the [cell](api.rst#ipysheet.easy.cell) function, we can create [Cell](api.rst#ipysheet.sheet.Cell) widgets that are directly added to the current sheet.

In [8]:
sheet = ipysheet.sheet(rows=3, columns=40, column_width=1) # change column_width
cell1 = ipysheet.cell(0, 0, 'Hello')
cell2 = ipysheet.cell(2, 0, 'World')
cell_value = ipysheet.cell(2,2, 42.)
sheet

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value='Hello'), Cell(colu…

In [47]:
cell1

Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value='hellow')

# Events
Using link or observe we can link widgets together, or attach event handlers


<div class="alert alert-warning">

**Note:** The examples below contain event handler written in Python that needs a running kernel, they will not work in the pure html documentation. They do work in binder!

</div>


In [25]:
import ipywidgets as widgets
sheet = ipysheet.sheet(rows=3, columns=2, column_headers=False, row_headers=False)
cell_a = ipysheet.cell(0, 1, 1, label_left='a') # add label to cell
cell_b = ipysheet.cell(1, 1, 2, label_left='b', numeric_format='0.00') # default numeric_format is '0.000'
cell_sum = ipysheet.cell(2, 1, 3, label_left='sum', read_only=True)

# create a slider linked to cell a
slider = widgets.FloatSlider(min=-10, max=10, description='a')
widgets.jslink((cell_a, 'value'), (slider, 'value'))

# changes in a or b should trigger this function
def calculate(change):
    cell_sum.value = cell_a.value + cell_b.value

cell_a.observe(calculate, 'value') # if cell_a.value changes, run calculate function 
cell_b.observe(calculate, 'value')


widgets.VBox([sheet, slider]) # sheet and slider can be splitted into separated cells in jupyter

# Cell ranges
Instead of referring to a single cell, we can also refer to cell ranges, rows and columns.

In [24]:
sheet = ipysheet.sheet(rows=10, columns=10)
row = ipysheet.row(0, [0, 1, 2, 3], background_color="red")
column = ipysheet.column(1, ["a", "b", "c", "d"], row_start=1, background_color="green")
cells = ipysheet.cell_range([["hi", "ola"], ["ciao", "bonjour"], ["hallo", "guten tag"]],
                            row_start=2, column_start=2, background_color="yellow")
sheet


Sheet(cells=(Cell(column_end=3, column_start=0, row_end=0, row_start=0, squeeze_column=False, style={'backgrou…

## Calculations
Since this is such a common pattern, a helper decorator [calculation](api.rst#ipysheet.easy.calculation) is provided, shortening the above code considerably.

In [7]:
import ipywidgets as widgets
sheet = ipysheet.sheet(rows=3, columns=2, column_headers=False, row_headers=False)
cell_a = ipysheet.cell(0, 1, 1, label_left='a')
cell_b = ipysheet.cell(1, 1, 2, label_left='b')
cell_sum = ipysheet.cell(2, 1, 3, label_left='sum', read_only=True)

# create a slider linked to cell a
slider = widgets.FloatSlider(min=-10, max=10, description='a')
widgets.jslink((cell_a, 'value'), (slider, 'value'))

@ipysheet.calculation(inputs=[cell_a, cell_b], output=cell_sum)
def calculate(a, b):
    return a + b

widgets.VBox([sheet, slider])

# Renderers
ipysheet is build on Handsontable, which allows [custom renderers](https://docs.handsontable.com/demo-custom-renderers.html), which we also support.

In [8]:
jscode_renderer_negative = """function (value) {
  return {
    backgroundColor: value < 0 ?  'red' : 'green'
  };
}
"""
ipysheet.renderer(code=jscode_renderer_negative, name='negative');

In [9]:
import random
s = ipysheet.sheet(rows=3, columns=4)
data = [[random.randint(-10, 10) for j in range(4)] for j in range(3)]
ipysheet.cell_range(data, renderer='negative')
s

Sheet(cells=(Cell(column_end=3, column_start=0, renderer='negative', row_end=2, row_start=0, squeeze_column=Fa…

If [flexx](http://flexx.readthedocs.io/en/stable/pyscript/index.html) is installed, Python code can be transpiled to JavaScript at runtime.

In [10]:
def renderer_negative(value):
    return {
        'backgroundColor': 'orange' if value < 0 else ''
    }
ipysheet.renderer(code=renderer_negative, name='negative_transpiled');

In [11]:
import random
s = ipysheet.sheet(rows=3, columns=4)
data = [[random.randint(-10, 10) for j in range(4)] for j in range(3)]
ipysheet.cell_range(data, renderer='negative_transpiled')
s

Sheet(cells=(Cell(column_end=3, column_start=0, renderer='negative_transpiled', row_end=2, row_start=0, squeez…

[api](api.rst)

# Plot 

In [14]:
import numpy as np
from traitlets import link
from ipywidgets import HBox
import bqplot.pyplot as plt
from ipysheet import sheet, cell, column

In [15]:
size = 18
scale = 100.
np.random.seed(0)
x_data = np.arange(size)
y_data = np.cumsum(np.random.randn(size)  * scale)

In [17]:
fig = plt.figure()
axes_options = {'x': {'label': 'Date', 'tick_format': '%m/%d'},
                'y': {'label': 'Price', 'tick_format': '0.0f'}}

scatt = plt.scatter(x_data, y_data, colors=['red'], stroke='black')
fig.layout.width = '70%'

In [18]:
fig

Figure(axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale())], fig_margin={'top':…

In [19]:
sheet1 = sheet(rows=size, columns=2)
x_column = column(0, x_data)
y_column = column(1, y_data)

In [22]:
link((scatt, 'x'), (x_column, 'value'))
link((scatt, 'y'), (y_column, 'value'))

HBox((fig, sheet1))

# Numpy back and forth 

In [24]:
import numpy as np
from ipysheet import from_array, to_array

arr = np.random.randn(6, 10)

sheet = from_array(arr)
sheet


Sheet(cells=(Cell(column_end=9, column_start=0, row_end=5, row_start=0, squeeze_column=False, squeeze_row=Fals…

In [25]:
arr = np.array([True, False, True])

sheet = from_array(arr)
sheet

Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format=None, row_end=2, row_start=0, squeeze_row=False…

In [26]:
to_array(sheet)

array([[ True],
       [False],
       [ True]])

# pandas back and forth 

In [27]:
import numpy as np
import pandas as pd
from ipysheet import from_dataframe, to_dataframe

dates = pd.date_range('20130101', periods=6)
df = pd.DataFrame(np.random.randn(6, 26), index=dates, columns=list(chr(ord('A') + i) for i in range(26)))

sheet = from_dataframe(df)
sheet

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=5, row_start=0, squeeze_row=False, type='numeric', val…

In [28]:
df2 = pd.DataFrame({'A': 1.,
                    'B': pd.Timestamp('20130102'),
                    'C': pd.Series(1, index=list(range(4)), dtype='float32'),
                    'D': np.array([False, True, False, False], dtype='bool'),
                    'E': pd.Categorical(["test", "train", "test", "train"]),
                    'F': 'foo'})

df2.loc[[0, 2], ['B']] = np.nan

sheet2 = from_dataframe(df2)
sheet2

Sheet(cells=(Cell(column_end=0, column_start=0, row_start=0, squeeze_row=False, type='numeric', value=[1.0, 1.…

In [29]:
df3 = to_dataframe(sheet2)
df3

,A,B,C,D,E,F
0,1.0,NaT,1.0,False,test,foo
1,1.0,2013-01-02,1.0,True,train,foo
2,1.0,NaT,1.0,False,test,foo
3,1.0,2013-01-02,1.0,False,train,foo


# Cell widget 

In [30]:
from ipywidgets import FloatSlider, IntSlider, Image
import ipysheet

In [31]:
slider = FloatSlider()
slider
sheet = ipysheet.sheet()

In [32]:
cell1 = ipysheet.cell(0, 0, slider, style={'min-width': '150px'})
cell3 = ipysheet.cell(2, 2, 42.)
cell_sum = ipysheet.cell(3, 2, 42.)

In [33]:
@ipysheet.calculation(inputs=[(cell1, 'value'), cell3], output=cell_sum)
def calculate(a, b):
    return a + b

sheet

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, style={'min-width': '150px'}, type='wi…

In [34]:
slider2 = IntSlider()
cell1.value = slider2
slider2
sheet = ipysheet.sheet()

column1 = ipysheet.column(0, [FloatSlider() for _ in range(5)])
column2 = ipysheet.column(1, [1.] * 5)

sheet

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=4, row_start=0, squeeze_row=False, type='widget', valu…

In [36]:
column1.value = column1.value[:-1] + [IntSlider()]
ipysheet.to_dataframe(sheet)
sheet = ipysheet.sheet()

column1 = ipysheet.column(0, [Image.from_file('./image.png') for _ in range(5)], style={'min-width': '60px', 'min-height': '50px'})
column2 = ipysheet.column(1, [1.] * 5)

sheet

FileNotFoundError: [Errno 2] No such file or directory: './image.png'

# format

In [37]:
import ipysheet
sheet = ipysheet.sheet()
cell0 = ipysheet.cell(0, 0, 0, numeric_format='0.0', type='numeric')
cell1 = ipysheet.cell(1, 0, "Hello", type='text')
cell2 = ipysheet.cell(0, 1, 0.1, numeric_format='0.000', type='numeric')
cell3 = ipysheet.cell(1, 1, 15.9, numeric_format='0.00', type='numeric')
cell4 = ipysheet.cell(2, 2, "02/14/2019", date_format='MM/DD/YYYY', type='date')

sheet

Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format='0.0', row_end=0, row_start=0, type='numeric', …